<a href="https://colab.research.google.com/github/fattie0926/StormNewsFetcher/blob/main/StormNewsFetcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StormNewsFetcher 使用說明
Developer：[fattie0926](https://github.com/fattie0926)

## 簡介
StormNewsFetcher 是一款運用 Python 和 BeautifulSoup 開發的新聞抓取工具，專門用於自動從[風傳媒](https://www.storm.mg/)網站抓取最新新聞。此工具通過解析網站的 HTML 結構來獲取新聞內容，並支援將內容透過 IFTTT 發送到 LINE 或其他平台。

## 功能
- **新聞抓取**：從風傳媒網站抓取最新新聞標題、內容及圖片。
- **字數限制**：提供選項限制新聞內容的字數，以適應不同平台的顯示需求。
- **自動發布**：透過 IFTTT Webhooks 功能，自動將新聞內容發送到 LINE 等平台。

## 原理
- 利用 `requests` 和 `BeautifulSoup` 對風傳媒網站進行 HTTP 請求和 HTML 內容解析。
- 提取所需的新聞元素，如標題、內文和圖片連結。
- 透過 IFTTT 的 Webhooks 服務，將新聞內容推送到指定的平台，如 LINE。

## 使用方式
1. 設定 IFTTT 的事件名稱和 Webhooks 密鑰。
2. 運行提供的 Python 函數來抓取風傳媒的最新新聞。
3. 使用 `send_to_IFTTT` 函數將抓取的內容發送到您的 LINE 或其他支援 IFTTT 的平台。

## 致謝
特別感謝所有使用和支持此項目的朋友們。如有任何建議或問題，歡迎提出。

---
# 原始碼
- 匯入所需的函式庫

In [ ]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

- 在[這裡](https://ifttt.com/maker_webhooks/settings)找到你的 Webhooks Key（URL 的最後一串亂碼）

In [ ]:
# IFTTT 設定
EVENT_NAME = 'enter_your_event_name'  # 在 IFTTT 中創建的 Event Name
YOUR_IFTTT_KEY = 'enter_your_key' # 修改為你的 Webhooks Key

IFTTT_WEBHOOKS_URL = f'https://maker.ifttt.com/trigger/{EVENT_NAME}/with/key/{YOUR_IFTTT_KEY}'

- 設定好各個需要用到的 Function

In [68]:
def fetch_content(url, word_limit=True, print_content=True):
    """
    輸入連結後，回傳新聞內容：標題+時間、文章內容、第一張圖片
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    title = soup.find("title").text[:-4]  # 獲取文章標題
    publish_time = soup.find("meta", itemprop="datePublished")['content']  # 獲取發佈時間
    if print_content: # 在 Console 中列印文章標題
        print(title)

    # 格式化發佈時間
    published_datetime = datetime.fromisoformat(publish_time)
    formatted_publish_time = published_datetime.strftime("%Y/%m/%d, %H:%M")
    title += "\n" + formatted_publish_time

    # 檢查是否有找到圖片網址
    image_link = soup.find("link", rel="image_src")
    image_url = image_link['href'] if image_link else '無圖片網址'

    # 擷取文章內容
    article = soup.find('article')
    article_content = ''
    if article:
        paragraphs = article.find_all('p', aid=True)
        for p in paragraphs:
            article_content += p.get_text(strip=True) + '\n\n'

    # 根據 word_limit 限制字數
    if word_limit and len(article_content) > 300:
        article_content = article_content[:300] + '⋯⋯\n\n點擊連結閱讀完整文章：'

    # 加上原文連結
    article_content += '\n' + url

    return {'title': title, 'article_content': article_content, 'image_url': image_url}

def fetch_latest_news_urls(n, homepage_url="https://www.storm.mg/"):
    """
    在網站上獲取最新的 n 篇新聞（最多七篇），並將網址以列表形式回傳
    """
    n = max(1, min(n, 7))  # 確保 n 在 1 到 7 之間

    response = requests.get(homepage_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    start_index = soup.decode_contents().find('<!-- for cards: card_noImg -->')
    end_index = soup.decode_contents().find('<!-- end of cards -->', start_index)
    cards_html = soup.decode_contents()[start_index:end_index]

    cards_soup = BeautifulSoup(cards_html, 'html.parser')
    card_links = cards_soup.find_all('a', class_='card_link link_title', href=True)

    return ['https://www.storm.mg' + a['href'] for a in card_links][:n]

def send_to_IFTTT(data, webhook_url=IFTTT_WEBHOOKS_URL, event_name=EVENT_NAME, key=YOUR_IFTTT_KEY):
    """
    輸入內容後，透過 Webhook 傳送至 IFTTT
    """
    data_to_send = {
        'value1': data['title'],
        'value2': data['article_content'].strip(),
        'value3': data['image_url']
    }

    requests.post(webhook_url.format(event=event_name, your_key=key), json=data_to_send)

# 使用範例
## 範例 1：取得單篇新聞內容，並傳送訊息到 LINE

In [69]:
url = "https://www.storm.mg/lifestyle/4912747"  # 修改為你想要的新聞連結

content_data = fetch_content(url, word_limit=True)
send_to_IFTTT(content_data)

獨》福源花生醬風波！香港消委會駁「拿過期的來驗」　業者疑「政治操作」：只相信台灣


## 範例 2：從風傳媒官網抓取最新的 n 篇新聞，並傳送到 LINE

In [ ]:
n = 3  # 指定抓取的新聞數量

for url in fetch_latest_news_urls(n):
    content = fetch_content(url, word_limit=True)
    send_to_IFTTT(content)

舊金山拜習會》與拜登見面已達目的　習近平可專注迫切內政問題
以色列國防軍宣稱「在加薩醫院發現哈瑪斯隧道」
離岸風電區塊3-2期選商新規定　想拿額外100MW得先拿到購售電合約
